# 🔧 Overview of the Steps:


*   Install dependencies







* Load the model using Unsloth

* Attach LoRA (parameter-efficient fine-tuning)

* Prepare dataset

* Apply the chat template

* Tokenize and format data

* Fine-tune using SFTTrainer

* (Optional) Mask user inputs for training

* Train

* Run inference



# 🛠️ 1. Install Required Libraries


In [ ]:
#(recommended by Unsloth):
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth


# 📦 2. Load LLaMA 3.2 3B from Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None  # Let it auto-detect best precision
load_in_4bit = True  # Saves memory

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",  # You can change to 1B version
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# 🧠 3. Prepare the Model for Training with LoRA


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)


Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# ✅ 4. *Prepare* Tokenizer Template


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",  # or "llama-3" (depends on model)
)


# ✅ 5. Prepare Data


In [ ]:
from datasets import load_dataset

dataset = load_dataset("mlabonne/FineTome-100k", split = "train")


Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

# ✅ 6. Format Prompts


In [ ]:
from unsloth.chat_templates import standardize_sharegpt

# Convert ShareGPT format to HF format
dataset = standardize_sharegpt(dataset)

# Apply chat template
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text": texts }

dataset = dataset.map(formatting_prompts_func, batched = True)


Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

# ✅ 7. Fine-tuning Setup with **SFTTrainer**

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,  # Change this for longer training
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

# ✅ 9. Train

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.358300
2,1.351900
3,1.785400
4,1.347500
5,1.378300
6,1.301400
7,1.097300
8,1.356700
9,1.049600
10,0.956700


TrainOutput(global_step=60, training_loss=1.0462127635876337, metrics={'train_runtime': 460.2096, 'train_samples_per_second': 1.043, 'train_steps_per_second': 0.13, 'total_flos': 6133779547865088.0, 'train_loss': 1.0462127635876337})

# ✅ 10. Inference (Try the Model!)



In [ ]:
FastLanguageModel.for_inference(model)

prompt = [
  {"role": "user", "content":
    '''Evaluate the child's answer.

Question: "If you have 5 apples and give 2 to your friend, how many do you have left?"
Answer: "3"

Please return the following JSON format:
{
  "correctness": "",
  "creativity": "",
  "creativity_reason": "",
  "reasoning": ""
}
'''
  }
]

input_text = tokenizer.apply_chat_template(prompt, tokenize = False, add_generation_prompt = True, return_tensors = None)

inputs = tokenizer(input_text, return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 128, temperature = 0.7, top_p = 0.95)

print(tokenizer.decode(outputs[0]))


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Evaluate the child's answer.

Question: "If you have 5 apples and give 2 to your friend, how many do you have left?"
Answer: "3"

Please return the following JSON format:
{
  "correctness": "",
  "creativity": "",
  "creativity_reason": "",
  "reasoning": ""
}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The child's answer is correct. They have 5 apples and gave 2 to their friend, so they have 3 apples left. The child's reasoning is clear and logical.

Correctness: 1

Creativity: 0

Creativity reason: The child did not have any creative ideas or alternative solutions.

Reasoning: The child used a simple and logical approach to solve the problem. They correctly identified the number of apples they had before and after giving some to their friend.<|eot_id|>
